# Purpose of Notebook


### This notebook intends to describe the process of mapping VIEWS predictions onto maps


In the following code, we you will (i) obtain data from the VIEWS API (see this [link](https://github.com/prio-data/views_api/blob/development/DataExploration/Outreach_Helper.ipynb) for an example noteboook), (ii) merge the data to a geom DataFrame for mapping purposes, (iii) define our VIEWS mapper function, (iv) allow user specifications, (v) map the data.   

In [13]:
# Mapper
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx
import sqlalchemy as sa

from views_dataviz.map import mapper, utils
from views_dataviz import color
from views_dataviz.map.presets import ViewsMap
from views_dataviz.map import utils
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# working on the function here
from mappy import Mapper2, vid2date

#To make an API call 
import requests
import json
import os

home = os.path.expanduser("~")

# Specify where you want to save the data, else it is saved to your desktop. 


path = home+'/' 
localgitpath = path+'views/views3/FCDO_predicting_fatalities/'


#This url is the base 
api_location = 'https://api.viewsforecasting.org/'
EndOfHistory = 515

### Obtaining data from the API

Please specify the data you would like to fetch from the API below. See this link for a guide on how to specify your own calls: [LINK](https://github.com/prio-data/views_api/blob/development/DataExploration/Outreach_Helper.ipynb)

In [2]:
run = 'fatalities001_2022_11_t01'
loa = 'pgm'
tv = 'sb' # or 'osv' or 'ns'
model = 'sc_pgm_sb_main'


url = api_location+ run+ '/'+ loa+ '/'+ tv+'/'+ model

print(url)

# Request function
r = requests.get(url)
PAGE_SIZE=1000
master_list=[]
r=requests.get(url)
page_data=r.json()
master_list+=page_data['data']
    
while page_data['next_page'] != '':
    r=requests.get(page_data['next_page'])
    page_data=r.json()
    master_list+=page_data['data']
        
df=pd.DataFrame(master_list)

df = df.rename(columns = {'pg_id':'priogrid_gid'})
df['step'] = df['month_id'] - EndOfHistory

df.to_csv(path+'latest_run.csv', sep=';', encoding='utf-8', index=True)

display(df)

https://api.viewsforecasting.org/fatalities001_2022_11_t01/pgm/sb/sc_pgm_sb_main


,priogrid_gid,month_id,sc_pgm_sb_main,step
0,62356,516,0.0001,1
1,79599,516,0.0001,1
2,79600,516,0.0001,1
3,79601,516,0.0001,1
4,80317,516,0.0001,1
...,...,...,...,...
471955,190496,551,0.0306,36
471956,190507,551,0.0024,36
471957,190508,551,0.0021,36
471958,190510,551,0.0009,36


### Defining the Map format

This cell defines the maps scales. You should only change numbers in this box if you are interested in changing the scale.

In [3]:
standard_scale = [np.log1p(0),np.log1p(3),np.log1p(10), np.log1p(30), np.log1p(100),  np.log1p(300)]#, np.log1p(1000), np.log1p(3000),  np.log1p(10000)]
standard_scale_labels = ['0', '3','10', '30','100', '300']#, '1000', '3000', '10000']

small_scale=[np.log1p(0),np.log1p(3),np.log1p(10), np.log1p(30), np.log1p(100),  np.log1p(300)]#, np.log1p(1000)]


small_scale_labels = ['0', '3','10', '30','100', '300']#, '1000']

small_scale_nolabels = ['', '','', '','', '', '']


In [14]:
#Gathering the relevant data

# get pgm geometries
gdf_base = gpd.read_parquet(localgitpath+ 'geometry/pgm_geometry.parquet')

# get cm geometries
gdf_c = gpd.read_parquet(localgitpath+ 'geometry/cm_geometry.parquet')
gdf_c = gdf_c.to_crs(4326)

gdf2 = gdf_base.copy()

df = df.set_index('priogrid_gid')
gdf2 = gdf2.set_index('priogrid_gid')

joint = pd.merge(df, gdf2, left_index=True, right_index=True, how='left')
joint = gpd.GeoDataFrame(joint, geometry="geom")
joint = joint.reset_index()

## The following cell defines your map 

This allows the plot to map only over the country/region of interest. To do specific you interests, build the box following the examples below. 

Example: 
1. country = [x min, x max, y min, y max] OR: 
2. country = [longitude min, longitude max, latitude min, latitude max]

A bbox can easily be built by following this link: [Click me for building boxes](http://bboxfinder.com/)

In [25]:
# Defines which maps you would like to produce (ex: step 1 = '1', step 36 = '36')
stepstoplot = [ 1, 12, 36]

# Define the country or region you you want to map here:
country_region = 'HoA'
country = 'Kenya'

#Build the bounding box here: 
kenya = [33.0, 43.0, -5.5, 6.0]
hoa = [21.533203, 52.734375, -6.140555, 22.350076]

### Example: Kenya

In [26]:
for step in stepstoplot:
        month = step + EndOfHistory
        gdf = joint.loc[joint['month_id']==month]
        m=Mapper2(
        width=10,
        height=10,
        frame_on=True,
        title= 'Ensemble predictions as of ' + vid2date(EndOfHistory+step) + ', ' + str(step) + ' months after last month with data',
        bbox= kenya, 
        ).add_layer(
        gdf=joint.loc[joint['month_id']==month],
        map_scale=standard_scale,
        cmap="rainbow",
        edgecolor="black",
        linewidth=0.2,
        column='sc_pgm_sb_main', 
        inform_colorbar=True
        )
        ax=m.ax
        fg=gdf_c.plot(ax=ax,edgecolor='black',linewidth=2,facecolor='None')
        figure=m.fig
        fontdict={'fontsize':25}
        figure.text(0.45,0.5,'KENYA',fontdict=fontdict,color='black')
        
        m.cbar.set_ticks(standard_scale)
        m.cbar.set_ticklabels(standard_scale_labels)

        m.save(f'{path}PredictionMap_{country}_pgm_ensemble_standard_scale_r{EndOfHistory}_m{month}.png')

### Horn of Africa

In [24]:
for step in stepstoplot:
        month = step + EndOfHistory
        gdf = joint.loc[joint['month_id']==month]
        m=Mapper2(
        width=10,
        height=10,
        frame_on=True,
        title='Ensemble predictions as of ' + vid2date(EndOfHistory+step) + ', ' + str(step) + ' months after last month with data',
        bbox=hoa,
        ).add_layer(
        gdf = joint.loc[joint['month_id']==month],
        map_scale=standard_scale,
        cmap="rainbow",
        edgecolor="black",
        linewidth=0.2,
        column='sc_pgm_sb_main', 
        inform_colorbar=True
        )
        ax=m.ax
        fg=gdf_c.plot(ax=ax,edgecolor='grey',linewidth=2.0,facecolor='None')
        figure=m.fig
        fontdict={'fontsize':15}
        figure.text(0.47,0.32,'KENYA',fontdict=fontdict,color='black')
        figure.text(0.48,0.5,'ETHIOPIA',fontdict=fontdict,color='black')
        figure.text(0.47,0.68,'ERITREA',fontdict=fontdict,color='black')
        figure.text(0.33,0.31,'UGANDA',fontdict=fontdict,color='black')
        figure.text(0.27,0.46,'S. SUDAN',fontdict=fontdict,color='black')
        figure.text(0.35,0.18,'TANZANIA',fontdict=fontdict,color='black')
        figure.text(0.68,0.52,'SOMALIA',fontdict=fontdict,color='black')
        figure.text(0.62,0.58,'DJIBOUTI',fontdict=fontdict,color='black')
        figure.text(0.71,0.66,'YEMEN',fontdict=fontdict,color='black')
        figure.text(0.2,0.3,'DRC',fontdict=fontdict,color='black')
        figure.text(0.28,0.25,'RWANDA',fontdict=fontdict,color='black')
        figure.text(0.28,0.22,'BURUNDI',fontdict=fontdict,color='black')
        figure.text(0.28,0.7,'SUDAN',fontdict=fontdict,color='black')
        
        m.cbar.set_ticks(standard_scale)
        m.cbar.set_ticklabels(standard_scale_labels)


        m.save(f'{path}PredictionMap_{country_region}_pgm_ensemble_standard_scale_r{EndOfHistory}_m{month}.png')